# Random Forest Project

Predicting diabetes

- In the previous project we saw how we could use a decision tree to predict data sets related to classification and regression. However, did you know that we can improve the prediction of a tree by using a random forest?

- As we have studied, a random forest is a grouping of trees generated with random portions of the data and with random criteria. This view would allow us to improve the effectiveness of the model when an individual tree is not sufficient.

- In this project you will focus on this idea by training the dataset to improve the accuracy.

#### Step 1: Loading the dataset

In [1]:
import pandas as pd

# Reading the processed dataset

train_data = pd.read_csv("/workspaces/machine-learning-random-forest-Juli-MM/data/processed/clean_train.csv")
test_data = pd.read_csv("/workspaces/machine-learning-random-forest-Juli-MM/data/processed/clean_test.csv")

train_data.head()

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1.0,149.0,68.0,29.3,0.349,42.0,1
1,0.0,140.0,65.0,42.6,0.431,24.0,1
2,5.0,122.0,86.0,34.7,0.290,33.0,0
3,10.0,161.0,68.0,25.5,0.326,47.0,1
4,3.0,150.0,76.0,21.0,0.207,37.0,0


#### Step 2: Build a random forest

In [2]:
# Separate predictors and target variable in training and test data:

X_train = train_data.drop(["Outcome"], axis = 1)
y_train = train_data["Outcome"]
X_test = test_data.drop(["Outcome"], axis = 1)
y_test = test_data["Outcome"]